<a href="https://colab.research.google.com/github/zhouchun0105/Citadel-Datathon-2022/blob/main/citadel_datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Diabetes_Data_1999_2008.csv')

In [ ]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [ ]:
df.replace('?', None)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),[150-175),1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),[150-175),1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),[150-175),1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),[150-175),2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [ ]:
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


# Data Cleaning

In [ ]:
#drop unused features
df.drop(["encounter_id", "patient_nbr", "weight", "payer_code", "medical_specialty", "change", "diabetesMed"], axis=1, inplace=True)

In [ ]:
df.dropna(axis=0, inplace=True)

In [ ]:
#process discharge_disposition_id
null_li = [18, 25, 26]
discharge_li = [11, 19, 20, 21]
alive_li = [1,2,3,4,5,6,7,8,9,10,12,13,14,15,16,17,22,23,24,30,27,28,29]  

df['discharge_disposition_id']=df.discharge_disposition_id.replace(null_li, "UNKNOWN")
df['discharge_disposition_id']=df.discharge_disposition_id.replace(discharge_li, "ALIVE")
df['discharge_disposition_id']=df.discharge_disposition_id.replace(alive_li, "EXPIRED")

df['discharge_disposition_id'].value_counts()

EXPIRED    95434
UNKNOWN     4680
ALIVE       1652
Name: discharge_disposition_id, dtype: int64

In [ ]:
#process admission_type_id
df['admission_type_id']=df.admission_type_id.replace({1:2 , 2:3, 3:4, 4:4, 5:5, 6:5, 7:1, 8:5})
df['admission_type_id'].value_counts()

2    53990
4    18879
3    18480
5    10396
1       21
Name: admission_type_id, dtype: int64

In [ ]:
#process readmitted
df['readmitted']=df.readmitted.replace({'NO': 0 , '>30': 1, '<30': 2})
df['readmitted'].value_counts()

In [ ]:
#process age
df['age']=df.age.replace({'[0-10)':0,'[10-20)':1, '[20-30)':2, '[30-40)':3, '[40-50)':4, '[50-60)':5,
       '[60-70)':6, '[70-80)':7, '[80-90)':8, '[90-100)':9})
df['age'].value_counts()

In [ ]:
#process admission_source_id
df['admission_source_id'] = df.admission_source_id.replace([1,2,3],'Referral')
df['admission_source_id'] = df.admission_source_id.replace([4,5,6,10,18,22,25,26],'Transfer')
df['admission_source_id'] = df.admission_source_id.replace([9,15,17,20,21],'Unknown')
df['admission_source_id'] = df.admission_source_id.replace([11,12,13,14,23,24],'New_Born')
df['admission_source_id'] = df.admission_source_id.replace({7:'ER', 8:'CLE', 19:'Readmission'})
df['admission_source_id'].value_counts()

In [ ]:
#diagnosis 1
mask = {'428':'Congestive heart failure','414': 'Dissection of aorta',\
                                '786':'Respiratory abnormality','410':'Acute myocardial infarction of anterolateral wall',\
                                '486':'Pneumonia','427':'Paroxysmal supraventricular tachycardia','491':'Simple chronic bronchitis',\
                                '715':'Osteoarthrosis','682':'Cellulitis and abscess of face',\
                                '434':'Cerebral thrombosis without mention of cerebral infarction'
                                }
df['diag_1']=df.diag_1.replace(mask)
labels = list(df.diag_1.unique())
new_label = [x for x in labels if x not in mask.values()]
df['diag_1'] = df.diag_1.replace(new_label,'Others')
df.diag_1.value_counts()

# Feature Engineering

In [ ]:
#cluster into [Treatment, Outcome, Demographic]
category = {'Treatment':['admission_type_id','num_lab_procedures','num_procedures','num_medications','diag_1','diag_2','diag_3','number_diagnoses','max_glu_serum','A1Cresult','metformin','repaglinide','nateglinide','chlorpropamide','glimepiride','acetohexamide','glipizide','glyburide','tolbutamide','pioglitazone','rosiglitazone','acarbose','miglitol','troglitazone','tolazamide','examide','citoglipton','insulin','glyburide-metformin','glipizide-metformin','glimepiride-pioglitazone','metformin-rosiglitazone','metformin-pioglitazone'], 
            'Demographic':['race', 'gender', 'age', 'admission_source_id'],
            'Outcome':['discharge_disposition_id','time_in_hospital', 'number_outpatient','number_emergency','number_inpatient','readmitted']}